In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path={'executable_path':ChromeDriverManager().install()}
browser=Browser('chrome',**executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\jlubr\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


# Chatham BottleKing Wine Scraper

In [3]:
# This url is specific to this location for bottleking.
url='https://chatham.bottleking.com/main.asp?request=search&type=w'
browser.visit(url)

In [4]:
# Create dataframe which will hold all the data scraped
wine_df=pd.DataFrame(columns=['name','country','region','state','varietal','price','description','merchant','address'])

merchant='Chatham BottleKing'
address='41 Watchung Ave, Chatham, NJ, 07928'

print("Scraping please be patient")

# Run this loop until all pages of the website have been visited
while True:
    html=browser.html
    soup=BeautifulSoup(html,'html.parser')
    wine_summary_boxes=soup.find_all('td',{'colspan':['2']})
    wine_summary_boxes.pop()
    
# Each page of the website has 10 wines which are inside summary boxes
    for box in wine_summary_boxes:
        
# Retreive information of the wine inside the summary box.
        name=box.find('a',{'class':['Srch-producttitle']}).text
        price=box.find('span',{'class':['RegularPrice']}).text
        description=box.find('span',{'class':['Srch-partialDesc']}).text
        wine=box.find('a',{'class':['Srch-producttitle']})

# Retreive url of wine to obtain more detailed information        
        url=wine.get('href')
        try:
            
# Visit detail page wine
            browser.visit(url)
            html=browser.html
            soup=BeautifulSoup(html,'html.parser')
            wineinfo=soup.find('div',{'class':['info_list']})

# Retreive the details of the wine
            details=wineinfo.find_all('span',{'class':['iteminfocat']})
    
# Retreive Country, Region, Sub-Region, Grape-Varietal of the wine...
#      Note: not all wines have a sub region so there is some coding which 
#            addresses this issue by use of the index variable. 
            bottle_info=wineinfo.find_all('a',{'class':['reglink']})
            index=0
            subregion=None
            for detail in details:
                if (detail.text =='Country:'):
                    country=bottle_info[index].text
                    index+=1
                if (detail.text =='Region:'):
                    region=bottle_info[index].text
                    index+=1
                if (detail.text =='Sub-Region:'):
                    subregion=bottle_info[2].text
                    index=3 
                if (detail.text =='Grape Varietal:'):
                    varietal=bottle_info[index].text
            browser.back()
        except:
            country=None
            region=None
            subregion=None
            variety=None
            browser.back()

# Append data retrieved into wine_df 
        wine_df=wine_df.append({
            'name':name,
            'country':country,
            'region':subregion,
            'state':region,
            'varietal':varietal,
            'price':price,
            'description':description,
            'merchant':merchant,
            'address':address
        },ignore_index=True)
    try:

# Click onto next page
        browser.links.find_by_partial_text('>>').click()
    except:
        break
browser.quit()
print('Scraping Complete')

# Write into csv.
wine_df.to_csv("chathambuyritecleaned.csv")

Scraping please be patient
Scraping Complete


In [5]:
wine_df


,name,country,region,state,varietal,price,description,merchant,address
0,Twisted Chardonnay,United States,None,California,Chardonnay,$8.99,Twisted is about having fun and enjoying the w...,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
1,Long Barn Pinot Noir 2018,United States,None,California,Pinot Noir,$12.99,One of the holy grail's of wine is finding a k...,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
2,Oyster Bay Sauvignon Blanc,New Zealand,None,Marlborough,Sauvignon Blanc,$15.99,The philosophy of Oyster Bay Marlborough is to...,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
3,Kim Crawford Sauvignon Blanc,New Zealand,None,Marlborough,Sauvignon Blanc,$17.99,Kim Crawford Wines started out in a small Auck...,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
4,Josh Sauvignon Blanc,United States,None,California,Sauvignon Blanc,$16.99,\nJoseph Carr was an award winning Sommelier a...,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
...,...,...,...,...,...,...,...,...,...
2061,Cossart Gordon Rainwtr Madeira,France,None,Bordeaux,Pinot Noir,$21.99,,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
2062,Haut Batailley Pauillac 18,France,None,Bordeaux,Pinot Noir,$79.99,,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
2063,Gilbert Rouge,Australia,None,Bordeaux,Pinot Noir,$19.99,,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
2064,Millton Chenin Blanc,New Zealand,None,Bordeaux,Pinot Noir,$36.99,,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
